Author: Ronny F. Pacheco Date: Jul 2024
Copyright: © 2024 Ronny Pacheco License: MIT License

---

MIT License

Copyright (c) 2024 Ronny Pacheco

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Needed mmodules

In [8]:
# Import needed packages
import pandas as pd
import os
import pickle
import json

In [7]:
## https://kioku-space.com/en/jupyter-skip-execution/
from IPython.core.magic import register_cell_magic # type: ignore

@register_cell_magic
def skip(line, cell):
    print('Skipping cell')
    if line and cell:
        pass
    return

# Pickle save

In [9]:
%%skip
# =============================================================================
# Save the variables
# =============================================================================
variables_dict = {
}

# =============================================================================
# main function
# =============================================================================
def data_save_load(option, dict_variables=None):
    """
    This function is used to save or load data for the jupyter notebook
    """
    path_folder = "ipynb_db"  # Folder to save variables
    os.makedirs(path_folder, exist_ok=True)  # Create folder if not exist
    notebook_name = os.path.basename(os.path.abspath(''))
    path_file = os.path.join(path_folder, f"{notebook_name}.variables.pkl") # Path to save the variables

    if option == "save":
        with open(path_file, "wb") as pickle_file:
            pickle.dump(dict_variables, pickle_file)
    elif option == "load":
        with open(path_file, "rb") as f:
            variables = pickle.load(f)
        # Now load the variables
        for pickle_key, pickle_value in variables.items():
            variables[pickle_key] = pickle_value

# =============================================================================
# Call the function
# =============================================================================
data_save_load(option="save",
               dict_variables=variables_dict)

Skipping cell


# 1. Prepare data

In [10]:
# Load the JSON dict into a Python dictionary
path_json_file = "./data/sider_gtf_relation.json"

with open(path_json_file, "r") as f:
    dict_sider_gtf = json.load(f)

In [11]:
# Check the size of the dictionary
print(f"Size of the dictionary: {len(dict_sider_gtf)} elements")

Size of the dictionary: 2117 elements


# 2. Create table

In [12]:
def gene_finder(sublist):
    """
    This function is used to find the gene name from the sublist without repetition
    """
    gene_name = ""
    gene_strand = ""
    if len(sublist) == 0:
        gene_name = "intergenic"
        gene_strand = None
    else:
        gene_list = []
        gene_strand = []
        for elem in sublist:
            if elem.get("gene_id") is not None:
                gene_list.append(elem.get("gene_id"))
                gene_strand.append(elem.get("strand"))
        gene_list = list(set(gene_list))  # Remove duplicates
        gene_name = ", ".join(gene_list)
        gene_strand = list(set(gene_strand))
        gene_strand = ", ".join(gene_strand)
    return gene_name, gene_strand

def transcript_finder(sublist):
    """
    This function is used to find the transcript name from the sublist without repetition
    """
    transcript_dict = {}
    transcript = False
    cds = False
    utr3 = False
    utr5 = False
    for elem in sublist:
        # Take transcript data
        if elem.get("transcript_id") is not None:
            # Create a key with the transcript_id if it does not exist already
            if transcript_dict.get(elem.get("transcript_id")) is None:
                transcript_dict[elem.get("transcript_id")] = []
            transcript = True
        # Now let's go for the CDS, 3utr and 5utr
        if elem.get("CDS") == True:
            transcript_dict[elem.get("parent_id")].append("CDS")
            cds = True
        if elem.get("3utr") == True:
            transcript_dict[elem.get("parent_id")].append("3utr")
            utr3 = True
        if elem.get("5utr") == True:
            transcript_dict[elem.get("parent_id")].append("5utr")
            utr5 = True
    
    return transcript_dict, transcript, cds, utr3, utr5

    

In [13]:
# Let's create a table where one column is the "sider name" and the other the "gene_name".
# If there si no gene_name, its intergenic region, so we will put "intergenic" in the gene_name column
list_data = []
for key, value in dict_sider_gtf.items():
    sider_name = key
    
    # Find the gen_id for each sider name
    gene_name, gene_strand = gene_finder(value)
    # list_data.append([sider_name, gene_name])

    # Find the transcript_id for each sider name
    transcript_name = ""
    transcript, cds, utr3, utr5 = False, False, False, False
    if gene_name != "intergenic":
        transcript_name, transcript, cds, utr3, utr5 = transcript_finder(value)
    else:
        transcript_name = "intergenic"
    list_data.append([sider_name, gene_name, gene_strand, transcript_name, transcript, cds, utr3, utr5])

# Create a pandas dataframe
df_sider_gtf = pd.DataFrame(list_data, columns=["sider_name", "gene_name", "gene_strand", "transcript_name", "transcript", "cds", "utr3", "utr5"])

In [14]:
print(df_sider_gtf.shape)
print(df_sider_gtf.dtypes)
df_sider_gtf.head()

(2117, 8)
sider_name         object
gene_name          object
gene_strand        object
transcript_name    object
transcript           bool
cds                  bool
utr3                 bool
utr5                 bool
dtype: object


,sider_name,gene_name,gene_strand,transcript_name,transcript,cds,utr3,utr5
0,sre_c01.10,intergenic,None,intergenic,False,False,False,False
1,sre_c01.20A,LINF_010005800,-,{'LINF_01T0005800': ['3utr']},True,False,True,False
2,sre_c01.30A,LINF_010006300,-,{'LINF_01T0006300': ['3utr']},True,False,True,False
3,sre_c01.40,LINF_010006600,-,{'LINF_01T0006600': ['3utr']},True,False,True,False
4,sre_c01.50A,LINF_010007050,-,{'LINF_01T0007050': []},True,False,False,False


# 3. Merge with the csv data

In [15]:
# Load data
og_data = pd.read_csv("../22.GFF_Creation/data_named/positive_data_named.csv", sep=",", header=0)
print(og_data.shape)
print(og_data.dtypes)
og_data.head()

(2117, 6)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
new_IDs    object
dtype: object


,sseqid,sstart,send,sstrand,sseq,new_IDs
0,LinJ.01,1,173,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...,sre_c01.10
1,LinJ.01,24093,24758,plus,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...,sre_c01.20A
2,LinJ.01,35371,35956,plus,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,sre_c01.30A
3,LinJ.01,39790,40595,plus,ATTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAG...,sre_c01.40
4,LinJ.01,54983,55547,plus,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,sre_c01.50A


In [10]:
# change the colum name "new_IDs" to "sider_name"
og_data.rename(columns={"new_IDs": "sider_name"}, inplace=True)

# Remove the "sstrand" column
og_data.drop(columns=["sstrand"], inplace=True)
og_data.head()

,sseqid,sstart,send,sseq,sider_name
0,LinJ.01,1,173,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...,sre_c01.10
1,LinJ.01,24093,24758,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...,sre_c01.20A
2,LinJ.01,35371,35956,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,sre_c01.30A
3,LinJ.01,39790,40595,ATTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAG...,sre_c01.40
4,LinJ.01,54983,55547,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...,sre_c01.50A


In [11]:
# Merge the two dataframes by the "sider_name" column
df_merged = pd.merge(df_sider_gtf, og_data, on="sider_name", how="inner")
print(df_merged.shape)
print(df_merged.dtypes)
df_merged.head()

(2117, 12)
sider_name         object
gene_name          object
gene_strand        object
transcript_name    object
transcript           bool
cds                  bool
utr3                 bool
utr5                 bool
sseqid             object
sstart              int64
send                int64
sseq               object
dtype: object


,sider_name,gene_name,gene_strand,transcript_name,transcript,cds,utr3,utr5,sseqid,sstart,send,sseq
0,sre_c01.10,intergenic,None,intergenic,False,False,False,False,LinJ.01,1,173,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...
1,sre_c01.20A,LINF_010005800,-,{'LINF_01T0005800': ['3utr']},True,False,True,False,LinJ.01,24093,24758,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...
2,sre_c01.30A,LINF_010006300,-,{'LINF_01T0006300': ['3utr']},True,False,True,False,LinJ.01,35371,35956,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...
3,sre_c01.40,LINF_010006600,-,{'LINF_01T0006600': ['3utr']},True,False,True,False,LinJ.01,39790,40595,ATTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAG...
4,sre_c01.50A,LINF_010007050,-,{'LINF_01T0007050': []},True,False,False,False,LinJ.01,54983,55547,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...


In [12]:
df_merged.rename(columns={
    "sseqid" : "sider_chrom",
    "sstart" : "sider_start",
    "send" : "sider_end",
    "sseq" : "sider_seq"
}, inplace=True)

df_merged.head()

,sider_name,gene_name,gene_strand,transcript_name,transcript,cds,utr3,utr5,sider_chrom,sider_start,sider_end,sider_seq
0,sre_c01.10,intergenic,None,intergenic,False,False,False,False,LinJ.01,1,173,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...
1,sre_c01.20A,LINF_010005800,-,{'LINF_01T0005800': ['3utr']},True,False,True,False,LinJ.01,24093,24758,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...
2,sre_c01.30A,LINF_010006300,-,{'LINF_01T0006300': ['3utr']},True,False,True,False,LinJ.01,35371,35956,ACTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...
3,sre_c01.40,LINF_010006600,-,{'LINF_01T0006600': ['3utr']},True,False,True,False,LinJ.01,39790,40595,ATTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAG...
4,sre_c01.50A,LINF_010007050,-,{'LINF_01T0007050': []},True,False,False,False,LinJ.01,54983,55547,ACTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGC...


In [13]:
# Export to CSV and Excel file
path_csv = "./data/sider_relationship.csv"
path_excel = "./data/sider_relationship.xlsx"

# Export to CSV
df_merged.to_csv(path_csv, index=False, sep=",", header=True)

# Export to Excel
df_merged.to_excel(path_excel, index=False, header=True)